In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [61]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import pandas as pd
import numpy as np
import random
random.seed(7)
import time
import pickle

In [3]:
then = time.time()
df = pd.read_csv('data/train/train.csv',dtype = {'acoustic_data': np.int16, 'time_to_failure': np.float64} ) # float32 is enough :)
now = time.time()
print(now-then)

121.95693302154541


In [4]:
ttf = df['time_to_failure'].values
index_start = np.nonzero(np.diff(ttf) > 0)[0] + 1
index_start = np.insert(index_start, 0, 0)
dict_df={}
for i in range(len(index_start)):
    if i<(len(index_start)-1):
        df_tmp=df[index_start[i]:index_start[i+1]]
    else:
        df_tmp=df[index_start[i]:]       
    dict_df["df"+str(i)]=df_tmp

In [5]:
len(dict_df)

17

In [6]:
def randomChoice(l):
    return random.randint(0, l - 1)

150000

In [20]:
def randomTrainingExample(df_dict):
    k = len(df_dict)
    num=randomChoice(k)
    #print(num)
    len_df=len(df_dict['df'+str(num)])
    idx_start=random.randint(0,len_df-149999)
    idx_end=idx_start+150000
    #print('Start Index:',idx_start)
    #print('End Index:',idx_end)
    df_tmp=df_dict['df'+str(num)]
    sample_x=df_tmp.iloc[idx_start:idx_end]['acoustic_data']
    sample_y=df_tmp.iloc[idx_start:idx_end]['time_to_failure']

    sample_x1=np.diff(sample_x)
    sample_y1=np.diff(sample_y)
    meanx1=np.mean(sample_x1)
    meany1=np.mean(sample_y1)
    sample_x1=np.append(sample_x1,meanx1)
    sample_y1=np.append( sample_y1,meany1)

    sample_x2=np.diff(sample_x1)
    sample_y2=np.diff(sample_y1)
    meanx2=np.mean(sample_x2)
    meany2=np.mean(sample_y2)
    sample_x2=np.append(sample_x2,meanx2)
    sample_y2=np.append(sample_y2,meany2)
  #   sample_y2=np.append(sample_y2,meany2)

    sample_x=np.array(sample_x)
    sample_y=np.array(sample_y)
    sample_x1=np.array(sample_x1)
    sample_y1=np.array(sample_y1)
    sample_x2=np.array(sample_x2)
    sample_y2=np.array(sample_y2)

    #print(sample_x.shape)
    #print(sample_x1.shape)
    #print(sample_x2.shape)
    xtable= [[ 0 for i in range(300) ] for j in range(500)]
    ytable= [[ 0 for i in range(300) ] for j in range(500)]
    for i in range(300):
        for j in range(500):
            x=[]
            x1=[]
            x2=[]
            x.append(sample_x[500*i+j])
            x1.append(sample_x1[500*i+j])
            x2.append(sample_x2[500*i+j])
            xtable[j][i]=x+x1+x2
    for i in range(300):
        for j in range(500):
            x=[]
            x1=[]
            x2=[]
            x.append(sample_y[500*i+j])
            #x1.append(sample_y1[500*i+j])
            #x2.append(sample_y2[500*i+j])
            ytable[j][i]=x
    uid =str(num)+'-'+str(idx_start)+'-'+str(idx_end)
    return uid,xtable,ytable



In [21]:
uid,xtrain,ytrain=randomTrainingExample(dict_df)

In [22]:
#a = np.array(xtrain)
#a[0][0]
uid

'7-46923717-47073717'

In [23]:
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)

In [24]:
xtrain= xtrain.astype(np.float32)
ytrain= ytrain.astype(np.float32)

In [25]:
xtrain

array([[[-3.0000000e+00, -3.0000000e+00,  9.0000000e+00],
        [ 5.0000000e+00,  0.0000000e+00,  3.0000000e+00],
        [ 1.0000000e+00,  0.0000000e+00, -4.0000000e+00],
        ...,
        [ 6.0000000e+00,  1.0000000e+00, -4.0000000e+00],
        [ 7.0000000e+00, -6.0000000e+00,  8.0000000e+00],
        [ 5.0000000e+00,  3.0000000e+00, -9.0000000e+00]],

       [[-6.0000000e+00,  6.0000000e+00, -8.0000000e+00],
        [ 5.0000000e+00,  3.0000000e+00, -5.0000000e+00],
        [ 1.0000000e+00, -4.0000000e+00,  3.0000000e+00],
        ...,
        [ 7.0000000e+00, -3.0000000e+00,  9.0000000e+00],
        [ 1.0000000e+00,  2.0000000e+00,  3.0000000e+00],
        [ 8.0000000e+00, -6.0000000e+00,  6.0000000e+00]],

       [[ 0.0000000e+00, -2.0000000e+00,  2.0000000e+00],
        [ 8.0000000e+00, -2.0000000e+00,  5.0000000e+00],
        [-3.0000000e+00, -1.0000000e+00,  3.0000000e+00],
        ...,
        [ 4.0000000e+00,  6.0000000e+00, -1.0000000e+01],
        [ 3.0000000e+00,  5.0

In [88]:
num_epochs = 200
batch_size = 100
learning_rate = 10

In [27]:
img_transform = transforms.Compose([
    transforms.ToTensor()
])


In [28]:
xtrainimg=img_transform(xtrain)
ytrainimg=img_transform(ytrain)

In [71]:
dict_final_x={}
dict_final_y={}

In [72]:
n_examples=17
now=time.time()
while(n_examples):
    while(uid in dict_final_x.keys()):
        uid,xtrain,ytrain=randomTrainingExample(dict_df)
    xtrain=np.array(xtrain)
    ytrain=np.array(ytrain)
    xtrain= xtrain.astype(np.float64)
    ytrain= ytrain.astype(np.float64)
    dict_final_x[uid]=xtrainimg
    dict_final_y[uid]=ytrainimg
    n_examples-=1
then=time.time()
print("Time Taken:",then-now)

Time Taken: 11.167648553848267


In [73]:
dict_final_x[uid].shape

torch.Size([3, 500, 300])

In [38]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=(2,0)),  # b,16, 168, 100
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 84, 50
            nn.Conv2d(16, 8, 3, stride=3, padding=(0,2)),  # b, 8,28 , 18
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2)  # b, 8, 14, 9
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 29, 19
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 87, 57
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 3, 2, stride=2, padding=1),  # b, 3,172, 112
            nn.ReLU(),
            nn.ConvTranspose2d(3, 1, 1, stride=3, padding=(7,17)),  # b, 1, 500, 300
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [90]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
weight_decay=1e-1)

In [40]:
#the custom dataloader
def get_x(uid_list):
    for i,uid in enumerate(uid_list):
        x=dict_final_x[uid]
        x=x.view(1,x.shape[0],x.shape[1],x.shape[2])
        if(i==0):
            out=x
        else:
            out = torch.cat((out, x), 0)
    return out
        
def get_y(uid_list):
    for i,uid in enumerate(uid_list):
        y=dict_final_y[uid]
        y=y.view(1,y.shape[0],y.shape[1],y.shape[2])
        if(i==0):
            out=y
        else:
            out = torch.cat((out, y), 0)
    return out
        

In [91]:
dataloader = DataLoader(list(dict_final_x.keys()), batch_size=batch_size, shuffle=True)

In [92]:
for epoch in range(num_epochs):
    for data in dataloader:
        img = get_x(data)
        img_out=get_y(data)
        img_out=img_out.cuda()
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img_out)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))

epoch [1/200], loss:14.4167
epoch [2/200], loss:9.2727
epoch [3/200], loss:9.2744
epoch [4/200], loss:8.6980
epoch [5/200], loss:8.7110
epoch [6/200], loss:8.7267
epoch [7/200], loss:8.7717
epoch [8/200], loss:8.5887
epoch [9/200], loss:9.2255
epoch [10/200], loss:9.2048
epoch [11/200], loss:9.1888
epoch [12/200], loss:9.2753
epoch [13/200], loss:19.9739
epoch [14/200], loss:19.8310
epoch [15/200], loss:9.5886
epoch [16/200], loss:8.7191
epoch [17/200], loss:8.8064
epoch [18/200], loss:8.9585
epoch [19/200], loss:10.0423
epoch [20/200], loss:9.4628
epoch [21/200], loss:9.2669
epoch [22/200], loss:9.1810
epoch [23/200], loss:9.0844
epoch [24/200], loss:8.8051
epoch [25/200], loss:8.4066
epoch [26/200], loss:8.5031
epoch [27/200], loss:8.4946
epoch [28/200], loss:8.8274
epoch [29/200], loss:11.4920
epoch [30/200], loss:8.9512
epoch [31/200], loss:8.4569
epoch [32/200], loss:8.4131
epoch [33/200], loss:8.4080
epoch [34/200], loss:8.4071
epoch [35/200], loss:8.8964
epoch [36/200], loss:9.6

In [63]:
with open('trainy_17000_3channel.pickle', 'wb') as handle:
    pickle.dump(dict_final_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [64]:
with open('trainy_17000_3channel.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(dict_final_y==b)

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [69]:
len(dict_final_y.keys())

18700

In [68]:
len(b.keys())

18700